# Step 1: Prepare and Validate the Dataset

### Download the dataset

We will work with the [CoEdIT dataset](https://huggingface.co/datasets/grammarly/coedit) of text editing examples (Raheja, et al). In each example, the user asks a writing assistant to rewrite text to suit a specific task (editing fluency, coherence, clarity, or style) and receives a response.


```
# This is formatted as code

{
 "_id": "57241",
 "task": "coherence",
 "src": "Make the text more coherent: It lasted for 60 minutes. It featured the three men taking questions from a studio audience.",
 "tgt": "Lasting for 60 minutes, it featured the three men taking questions from a studio audience."
}

{
 "_id": "69028",
 "task": "clarity",
 "src": "Make the sentence clearer: URLe Lilanga (1934 27 June 2005) was a Tanzanian painter and sculptor, active from the late 1970s and until the early years of the 21st century.",
 "tgt": "URLe Lilanga (1934 27 June 2005) was a Tanzanian painter and sculptor, active from the late 1970s and until the early 21st century."
}

```

We will use the src and tgt fields from each example, which correspond to the user’s prompt and the writing assistant’s response, respectively. Instead of using the full dataset, we will use a subset focused on making text coherent: 927 total conversations.

To format the dataset for the Python SDK, we create a .jsonl where each JSON object is a conversation containing a series of messages.

A System message in the beginning, acting as the preamble that guides the whole conversation
Multiple pairs of User and Chatbot messages, representing the conversation that takes place between a human user and a chatbot. Here is a preview of the prepared dataset:

```
{'messages':
 [{'role': 'System',
   'content': 'You are a writing assistant that helps the user write coherent text.'
  },
  {'role': 'User',
   'content': 'Make the text more coherent: It lasted for 60 minutes. It featured the three men taking questions from a studio audience.'
  },
  {'role': 'Chatbot',
   'content': 'Lasting for 60 minutes, it featured the three men taking questions from a studio audience.'
  }
 ]
}

```



In [ ]:
! pip install cohere jsonlines -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.6 MB/s eta 0:00:00


In [ ]:
import cohere
import os
import json
import jsonlines

In [ ]:
co = cohere.ClientV2("VKxoCS7wrKZgmI7GMtm0OTpDYNNoFGDw3mKEdQQT")

In [ ]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Download the dataset
from datasets import load_dataset

! wget "https://huggingface.co/datasets/grammarly/coedit/resolve/main/train.jsonl"

--2024-12-03 13:55:14--  https://huggingface.co/datasets/grammarly/coedit/resolve/main/train.jsonl
Resolving huggingface.co (huggingface.co)... 3.169.137.19, 3.169.137.119, 3.169.137.5, ...
Connecting to huggingface.co (huggingface.co)|3.169.137.19|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/30/91/3091c2c741f77a2f5aa8986b13e4fb2c3658ab3ebc30ecaa5f6890e60939bdf9/2913249158d6a178dc638e870212ff8a432d128eb6b4bdbe969ee805e6063ce3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27train.jsonl%3B+filename%3D%22train.jsonl%22%3B&Expires=1733493314&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMzQ5MzMxNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8zMC85MS8zMDkxYzJjNzQxZjc3YTJmNWFhODk4NmIxM2U0ZmIyYzM2NThhYjNlYmMzMGVjYWE1ZjY4OTBlNjA5MzliZGY5LzI5MTMyNDkxNThkNmExNzhkYzYzOGU4NzAyMTJmZjhhNDMyZDEyOGViNmI0YmRiZTk2OWVlODA1ZTYwNjNjZTM%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0

### Get a subset of the dataset

Instead of using the full dataset, we will use a subset focused on making text coherent: 927 total conversations.

In [ ]:
# we will use subset of the dataset focused on making text more coherent
phrase = "coherent"

# instantiate python list where we will store correct subset of dataset
dataset_list = []

# create subset of dataset
with jsonlines.open('train.jsonl') as f:
    for line in f.iter():
        if phrase in line['src'].split(":")[0]:
            dataset_list.append(line)

# Split data into training and test
dataset_list_train = dataset_list[:800]
dataset_list_test = dataset_list[800:]

print("Total number of examples:", len(dataset_list))
print("Number of examples in training set:", len(dataset_list_train))
print("Number of examples in the test set:", len(dataset_list_test))

Total number of examples: 927
Number of examples in training set: 800
Number of examples in the test set: 127


### Preview the dataset

We will use the `src` and `tgt` fields from each example, which correspond to the user’s prompt and the writing assistant’s response, respectively.

In [ ]:
# print the first ten prompts and corresponding responses
for item in dataset_list_train[:10]:
    print(item["src"])
    print(item["tgt"])
    print("-"*50)

Make the text coherent: The Bank's main strategy is to further expand its network and increase its lending activities with particular focus on the SME sector. The EBRD helps Bank, by developing and financing Bank's portfolio of and strengthening the bank's funding base.
The Bank's main strategy is to further expand its network and increase its lending activities with particular focus on the SME sector. The EBRD helps Union Bank, by developing and financing its portfolio of and strengthening the bank's funding base.
--------------------------------------------------
Make the text coherent: It was not illegal under international law ; captured foreign sailors were released. Confederates went to prison camps.
It was not illegal under international law ; captured foreign sailors were released, while Confederates went to prison camps.
--------------------------------------------------
Make the text coherent: The Union blockade was a powerful weapon that eventually ruined the Southern econom

### Prepare the dataset for Cohere's Chat endpoint

To format the dataset for the Chat endpoint, we create a `.jsonl` where each JSON object is a conversation containing a series of messages.
- A `System` message in the beginning that guides the whole conversation
- Multiple pairs of `User` and `Chatbot` messages, representing the conversation that takes place between a human user and a chatbot

In [ ]:
# arranges the data to suit Cohere's format
def create_chat_ft_data(system_message, user_message, chatbot_message):
    formatted_data = {
        "messages": [
            {
                "role": "System",
                "content": system_message
            },
            {
                "role": "User",
                "content": user_message
            },
            {
                "role": "Chatbot",
                "content": chatbot_message
            }
        ]
    }

    return formatted_data

system_message = "You are a writing assistant that helps the user write coherent text."

In [ ]:
# creates jsonl file from list of examples
def create_jsonl_from_list(file_name, dataset_segment, system_message):
    path = f'{file_name}.jsonl'
    if not os.path.isfile(path):
        with open(path, 'w+') as file:
            for item in dataset_segment:
                user_message = item["src"]
                chatbot_message = item["tgt"]
                formatted_data = create_chat_ft_data(system_message, user_message, chatbot_message)
                file.write(json.dumps(formatted_data) + '\n')
            file.close()

# Create training jsonl file
file_name = "coedit_coherence_train"
create_jsonl_from_list(file_name, dataset_list_train, system_message)

# List the first 3 items in the JSONL file
with jsonlines.open(f'{file_name}.jsonl') as f:
    [print(line) for _, line in zip(range(3), f)]

{'messages': [{'role': 'System', 'content': 'You are a writing assistant that helps the user write coherent text.'}, {'role': 'User', 'content': "Make the text coherent: The Bank's main strategy is to further expand its network and increase its lending activities with particular focus on the SME sector. The EBRD helps Bank, by developing and financing Bank's portfolio of and strengthening the bank's funding base."}, {'role': 'Chatbot', 'content': "The Bank's main strategy is to further expand its network and increase its lending activities with particular focus on the SME sector. The EBRD helps Union Bank, by developing and financing its portfolio of and strengthening the bank's funding base."}]}
{'messages': [{'role': 'System', 'content': 'You are a writing assistant that helps the user write coherent text.'}, {'role': 'User', 'content': 'Make the text coherent: It was not illegal under international law ; captured foreign sailors were released. Confederates went to prison camps.'}, {

## Step 3: Use/Evaluate the Fine-Tuned Model



### With Test Data

When you're ready to use the fine-tuned model, navigate to the API tab. There, you'll see the model ID that you should use when calling `co.chat()`.

<img src="https://files.readme.io/82c726e-get_model_id.png">

In the following code, we supply the first three messages from the test dataset to both the pre-trained and fine-tuned models for comparison.

In [ ]:
for item in dataset_list_test[:1]:
    # User prompt
    user_message = item["src"]
    # Desired/target response from dataset
    tgt_message = item["tgt"]

    # Get default model response
    response_pretrained = co.chat(
        model="command-r-plus",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ]
    )

    # Get fine-tuned model response
    response_finetuned = co.chat(
        model="ca08b7a8-eaad-4fd1-b1fa-2427cb2ce1a6-ft",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ]
    )

    print(f"User: {user_message}","\n-----")
    print(f"Desired response: {tgt_message}","\n-----")
    # Accessing the text from the first element of the list
    print(f"Default model's response: {response_pretrained.message.content[0].text}","\n-----")
    # Accessing the text from the first element of the list
    print(f"Fine-tuned model's response: {response_finetuned.message.content[0].text}")
    print("-"*100,"\n\n")

User: Make the text more coherent: We do know that at the end of the Muromachi period it stopped appearing in written records. That Muromachi burned down many times, the last we know of in 1405. 
-----
Desired response: We do know that at the end of the Muromachi period it stopped appearing in written records and that it burned down many times, the last we know of in 1405. 
-----
Default model's response: We do know that at the end of the Muromachi period, specifically after 1405 (the year of the last recorded fire), it stopped appearing in written records. This may be due to the fact that Muromachi burned down many times. 
-----
Fine-tuned model's response: We do know that at the end of the Muromachi period it stopped appearing in written records, although the district burned down many times, the last we know of in 1405.
---------------------------------------------------------------------------------------------------- 




In [ ]:
model = "ca08b7a8-eaad-4fd1-b1fa-2427cb2ce1a6-ft"

def run_chat(user_message, messages=None):
    # Initialize messages as an empty list if None is passed
    if messages is None:
        messages = []

    # Check if system message already exists
    if not any(m.get('role') == 'system' for m in messages):
        messages.append({"role": "system", "content": system_message})

    # Generate response
    response = co.chat(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ]
    )

    # Print the response
    print(response.message.content)

    # Append the turn to the chat history
    messages.extend([
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": response.message.content[0].text}
    ])

    return messages

In [ ]:
messages = run_chat("Hello")

[TextAssistantMessageResponseContentItem(type='text', text='Hello! How can I help you?')]


In [ ]:
messages = run_chat("I'm fine. Can I ask you for help with some tasks?", messages)

[TextAssistantMessageResponseContentItem(type='text', text='Sure, what do you need help with?')]


In [ ]:
messages = run_chat("Make this more coherent: Manuel now has to decide-will he let his best friend be happy with her Prince Charming. Or will he fight for the love that has kept him alive for the last 16 years?", messages)

[TextAssistantMessageResponseContentItem(type='text', text='Manuel now has to decide-will he let his best friend be happy with her Prince Charming, or will he fight for the love that has kept him alive for the last 16 years?')]


In [ ]:
messages = run_chat("Help me with this one - She left Benaras. Conditions back home were bad.", messages)

[TextAssistantMessageResponseContentItem(type='text', text='Leaving Benaras, conditions back home were bad.')]
